Perform any cleaning, exploratory analysis, and/or visualizations to use the provided data for this analysis (a few sentences/plots describing your approach will suffice). What fraction of the observed users were retained?

Build a predictive model to help Ultimate determine whether or not a user will be active in their 6th month on the system. Discuss why you chose your approach, what alternatives you considered, and any concerns you have. How valid is your model? Include any key indicators of model performance.

Briefly discuss how Ultimate might leverage the insights gained from the model to improve its longterm rider retention (again, a few sentences will suffice).

In [1]:
import json
import seaborn as sns
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [2]:
with open('ultimate_data_challenge.json') as f:
    data=json.load(f)
df = pd.DataFrame(data)

In [3]:
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 12 columns):
city                      50000 non-null object
trips_in_first_30_days    50000 non-null int64
signup_date               50000 non-null object
avg_rating_of_driver      41878 non-null float64
avg_surge                 50000 non-null float64
last_trip_date            50000 non-null object
phone                     49604 non-null object
surge_pct                 50000 non-null float64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
avg_dist                  50000 non-null float64
avg_rating_by_driver      49799 non-null float64
dtypes: bool(1), float64(6), int64(1), object(4)
memory usage: 4.2+ MB


There are some missing values in avg rating, phone type and avg rating by driver

In [5]:
df.describe()

,trips_in_first_30_days,avg_rating_of_driver,avg_surge,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver
count,50000.000000,41878.000000,50000.000000,50000.000000,50000.000000,50000.000000,49799.000000
mean,2.278200,4.601559,1.074764,8.849536,60.926084,5.796827,4.778158
std,3.792684,0.617338,0.222336,19.958811,37.081503,5.707357,0.446652
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,4.300000,1.000000,0.000000,33.300000,2.420000,4.700000
50%,1.000000,4.900000,1.000000,0.000000,66.700000,3.880000,5.000000
75%,3.000000,5.000000,1.050000,8.600000,100.000000,6.940000,5.000000
max,125.000000,5.000000,8.000000,100.000000,100.000000,160.960000,5.000000


In [6]:
# lets replace the empty rows with the mean values
df.avg_rating_of_driver.fillna(4.6, inplace=True)
df.avg_rating_by_driver.fillna(4.7, inplace=True)

In [7]:
df.describe()

,trips_in_first_30_days,avg_rating_of_driver,avg_surge,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,2.278200,4.601306,1.074764,8.849536,60.926084,5.796827,4.777844
std,3.792684,0.564977,0.222336,19.958811,37.081503,5.707357,0.445781
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,4.500000,1.000000,0.000000,33.300000,2.420000,4.700000
50%,1.000000,4.700000,1.000000,0.000000,66.700000,3.880000,5.000000
75%,3.000000,5.000000,1.050000,8.600000,100.000000,6.940000,5.000000
max,125.000000,5.000000,8.000000,100.000000,100.000000,160.960000,5.000000


In [8]:
type(df.last_trip_date[0])

str

So we should convert string to a datetime object

In [9]:
df.last_trip_date = pd.to_datetime(df.last_trip_date)

In [10]:
type(df.last_trip_date[0])

pandas._libs.tslibs.timestamps.Timestamp

Lets apply a heatmap to see the correlation among features

In [11]:
df.last_trip_date.describe()

count                   50000
unique                    182
top       2014-06-29 00:00:00
freq                     2036
first     2014-01-01 00:00:00
last      2014-07-01 00:00:00
Name: last_trip_date, dtype: object

In [12]:
df.signup_date.min()

'2014-01-01'

In [13]:
df.signup_date.max()

'2014-01-31'

We can see that the last trip of the data set was done on 2014-07-01, so using ultimate till 2014-06-01 means being active, finding people registered before this date and used the system till this date would give us the target data (active or not active)

In [14]:
df.signup_date = pd.to_datetime(df.signup_date)
df.last_trip_date = pd.to_datetime(df.last_trip_date)

In [15]:
type(df.last_trip_date)

pandas.core.series.Series

In [16]:
# lets find active users

df['active']=np.nan
for i in range(len(df.last_trip_date)):
    if ((df['last_trip_date'][i])>=(df['signup_date'][i]+pd.Timedelta('150 days'))):
        df['active'][i]=1
    else:
        df['active'][i]=0

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
df.head()

,city,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,phone,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,active
0,King's Landing,4,2014-01-25,4.7,1.10,2014-06-17,iPhone,15.4,True,46.2,3.67,5.0,0.0
1,Astapor,0,2014-01-29,5.0,1.00,2014-05-05,Android,0.0,False,50.0,8.26,5.0,0.0
2,Astapor,3,2014-01-06,4.3,1.00,2014-01-07,iPhone,0.0,False,100.0,0.77,5.0,0.0
3,King's Landing,9,2014-01-10,4.6,1.14,2014-06-29,iPhone,20.0,True,80.0,2.36,4.9,1.0
4,Winterfell,14,2014-01-27,4.4,1.19,2014-03-15,Android,11.8,False,82.4,3.13,4.9,0.0


In [18]:
df.active.value_counts()          

0.0    37286
1.0    12714
Name: active, dtype: int64

fig = plt.figure()
ax = plt.subplot(111)
corr = df.corr()
ax = sns.heatmap(corr)
plt.show()

In [19]:
df.corr()

,trips_in_first_30_days,avg_rating_of_driver,avg_surge,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,active
trips_in_first_30_days,1.000000,-0.010861,-0.001841,0.005720,0.112210,0.050388,-0.136329,-0.038774,0.210011
avg_rating_of_driver,-0.010861,1.000000,-0.021660,-0.003283,-0.001879,0.012609,0.028373,0.101709,-0.007845
avg_surge,-0.001841,-0.021660,1.000000,0.793582,-0.078791,-0.110071,-0.081491,0.010208,0.006780
surge_pct,0.005720,-0.003283,0.793582,1.000000,-0.106861,-0.144918,-0.104414,0.019719,0.017825
ultimate_black_user,0.112210,-0.001879,-0.078791,-0.106861,1.000000,0.035998,0.032310,0.009630,0.163169
weekday_pct,0.050388,0.012609,-0.110071,-0.144918,0.035998,1.000000,0.101652,0.020511,0.007811
avg_dist,-0.136329,0.028373,-0.081491,-0.104414,0.032310,0.101652,1.000000,0.079742,-0.092038
avg_rating_by_driver,-0.038774,0.101709,0.010208,0.019719,0.009630,0.020511,0.079742,1.000000,-0.021164
active,0.210011,-0.007845,0.006780,0.017825,0.163169,0.007811,-0.092038,-0.021164,1.000000


It looks like, for the feature active, trips in the first 30 days and ultimate black user has the biggest correlation compared to other features. 

For nonnumeric features, we should use get dummies method

In [20]:
df_dummies=df[['city','phone']]
dummy=pd.get_dummies(df_dummies)
dffinal=pd.concat([df,dummy], axis=1)

In [21]:
dffinal.drop('city', axis=1, inplace=True)

In [22]:
dffinal.drop('phone', axis=1, inplace=True)

In [23]:
dffinal.head()

,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,surge_pct,ultimate_black_user,weekday_pct,avg_dist,avg_rating_by_driver,active,city_Astapor,city_King's Landing,city_Winterfell,phone_Android,phone_iPhone
0,4,2014-01-25,4.7,1.10,2014-06-17,15.4,True,46.2,3.67,5.0,0.0,0,1,0,0,1
1,0,2014-01-29,5.0,1.00,2014-05-05,0.0,False,50.0,8.26,5.0,0.0,1,0,0,1,0
2,3,2014-01-06,4.3,1.00,2014-01-07,0.0,False,100.0,0.77,5.0,0.0,1,0,0,0,1
3,9,2014-01-10,4.6,1.14,2014-06-29,20.0,True,80.0,2.36,4.9,1.0,0,1,0,0,1
4,14,2014-01-27,4.4,1.19,2014-03-15,11.8,False,82.4,3.13,4.9,0.0,0,0,1,1,0


In [24]:
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 16 columns):
trips_in_first_30_days    50000 non-null int64
signup_date               50000 non-null datetime64[ns]
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
last_trip_date            50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
ultimate_black_user       50000 non-null bool
weekday_pct               50000 non-null float64
avg_dist                  50000 non-null float64
avg_rating_by_driver      50000 non-null float64
active                    50000 non-null float64
city_Astapor              50000 non-null uint8
city_King's Landing       50000 non-null uint8
city_Winterfell           50000 non-null uint8
phone_Android             50000 non-null uint8
phone_iPhone              50000 non-null uint8
dtypes: bool(1), datetime64[ns](2), float64(7), int64(1), uint8(5)
memory usage: 4.1 MB


The only non-numeric data is black_user feature which is a boolean, we should change it ot 0/1 integers

In [25]:
blackuser=[]
for i in range(len(dffinal)):
    if dffinal['ultimate_black_user'][i]==True:
        blackuser.append(1)
    else:
        blackuser.append(0)

In [26]:
dffinal['ub_user']=blackuser

In [27]:
dffinal.drop('ultimate_black_user',axis=1, inplace=True)

In [28]:
dffinal.head()

,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver,active,city_Astapor,city_King's Landing,city_Winterfell,phone_Android,phone_iPhone,ub_user
0,4,2014-01-25,4.7,1.10,2014-06-17,15.4,46.2,3.67,5.0,0.0,0,1,0,0,1,1
1,0,2014-01-29,5.0,1.00,2014-05-05,0.0,50.0,8.26,5.0,0.0,1,0,0,1,0,0
2,3,2014-01-06,4.3,1.00,2014-01-07,0.0,100.0,0.77,5.0,0.0,1,0,0,0,1,0
3,9,2014-01-10,4.6,1.14,2014-06-29,20.0,80.0,2.36,4.9,1.0,0,1,0,0,1,1
4,14,2014-01-27,4.4,1.19,2014-03-15,11.8,82.4,3.13,4.9,0.0,0,0,1,1,0,0


In [29]:
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 16 columns):
trips_in_first_30_days    50000 non-null int64
signup_date               50000 non-null datetime64[ns]
avg_rating_of_driver      50000 non-null float64
avg_surge                 50000 non-null float64
last_trip_date            50000 non-null datetime64[ns]
surge_pct                 50000 non-null float64
weekday_pct               50000 non-null float64
avg_dist                  50000 non-null float64
avg_rating_by_driver      50000 non-null float64
active                    50000 non-null float64
city_Astapor              50000 non-null uint8
city_King's Landing       50000 non-null uint8
city_Winterfell           50000 non-null uint8
phone_Android             50000 non-null uint8
phone_iPhone              50000 non-null uint8
ub_user                   50000 non-null int64
dtypes: datetime64[ns](2), float64(7), int64(2), uint8(5)
memory usage: 4.4 MB


In [32]:
dffinal.columns

Index(['trips_in_first_30_days', 'signup_date', 'avg_rating_of_driver',
       'avg_surge', 'last_trip_date', 'surge_pct', 'weekday_pct', 'avg_dist',
       'avg_rating_by_driver', 'active', 'city_Astapor', 'city_King's Landing',
       'city_Winterfell', 'phone_Android', 'phone_iPhone', 'ub_user'],
      dtype='object')

In [33]:
dffinal.columns=['trips_in_first_30_days', 'signup_date', 'avg_rating_of_driver',
       'avg_surge', 'last_trip_date', 'surge_pct', 'weekday_pct', 'avg_dist',
       'avg_rating_by_driver', 'active', 'city_Astapor', 'city_Kings Landing',
       'city_Winterfell', 'phone_Android', 'phone_iPhone', 'ub_user']

### Predictive Modeling

In [30]:
#Import necessary libraries

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV



In [35]:
dffinal.head()

,trips_in_first_30_days,signup_date,avg_rating_of_driver,avg_surge,last_trip_date,surge_pct,weekday_pct,avg_dist,avg_rating_by_driver,active,city_Astapor,city_Kings Landing,city_Winterfell,phone_Android,phone_iPhone,ub_user
0,4,2014-01-25,4.7,1.10,2014-06-17,15.4,46.2,3.67,5.0,0.0,0,1,0,0,1,1
1,0,2014-01-29,5.0,1.00,2014-05-05,0.0,50.0,8.26,5.0,0.0,1,0,0,1,0,0
2,3,2014-01-06,4.3,1.00,2014-01-07,0.0,100.0,0.77,5.0,0.0,1,0,0,0,1,0
3,9,2014-01-10,4.6,1.14,2014-06-29,20.0,80.0,2.36,4.9,1.0,0,1,0,0,1,1
4,14,2014-01-27,4.4,1.19,2014-03-15,11.8,82.4,3.13,4.9,0.0,0,0,1,1,0,0


In [39]:
# train test split

y = dffinal['active']
X = dffinal[['trips_in_first_30_days', 'avg_rating_of_driver',
       'avg_surge', 'surge_pct', 'weekday_pct', 'avg_dist',
       'avg_rating_by_driver', 'city_Astapor', 'city_Kings Landing',
       'city_Winterfell', 'phone_Android', 'phone_iPhone', 'ub_user']]

## Logistic Regression

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state=42)

logreg = LogisticRegression()

# Fit the classifier to the training data
logreg.fit(X_train,y_train)

# Predict the labels of the test set: y_pred
y_pred_lr = logreg.predict(X_test)

# Compute and print the confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


[[14183   765]
 [ 3939  1113]]
              precision    recall  f1-score   support

         0.0       0.78      0.95      0.86     14948
         1.0       0.59      0.22      0.32      5052

    accuracy                           0.76     20000
   macro avg       0.69      0.58      0.59     20000
weighted avg       0.73      0.76      0.72     20000



In [50]:
print('Accuracy :', metrics.accuracy_score(y_test, y_pred_lr))

Accuracy : 0.7648


## Random Forest

In [51]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

rf = RandomForestClassifier(random_state=0, n_jobs=-1, class_weight="balanced", n_estimators = 200)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Accuracy :', metrics.accuracy_score(y_test, y_pred_rf))


Accuracy : 0.77175


In [56]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
avg_dist,0.297174
weekday_pct,0.146529
avg_rating_by_driver,0.113555
trips_in_first_30_days,0.088768
avg_rating_of_driver,0.087223
surge_pct,0.084318
avg_surge,0.071624
city_Kings Landing,0.030461
ub_user,0.023333
phone_iPhone,0.017638


## XGBOOST

In [52]:

from xgboost import XGBClassifier

xg_boost = XGBClassifier()
xg_boost.fit(X_train, y_train)
y_pred_xg_boost = xg_boost.predict(X_test)
print('Accuracy :', metrics.accuracy_score(y_test, y_pred_xg_boost))


Accuracy : 0.79835


In [57]:
feature_importances = pd.DataFrame(xg_boost.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
surge_pct,0.217801
avg_rating_by_driver,0.182936
phone_Android,0.115846
weekday_pct,0.103936
city_Kings Landing,0.100860
phone_iPhone,0.079691
ub_user,0.069262
city_Astapor,0.047713
trips_in_first_30_days,0.026098
avg_rating_of_driver,0.021607


## Conclusion

We have built some classification models to predict whether customers will be retained or not using classification algorithms.  We initially used logistic regression because it is a good predictor of binary classification. XGBOOST model gave the best accuracy result (0.80) with the features we have seleceted. For the random forest classifier, the factors that heavily influence rider retention are avarage distance, weekday percentage and avarage rating by the driver. For the XGBOOST model, highest features are surge percentage, avarage driver rating and weekday percentage